# Imports

In [1]:
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import pandas as pd
import duckdb

import asyncio

In [2]:
#print(sys.path)

____

___

In [3]:
filename = "data/your_path_here.duckdb"

# Read From duckbd

In [4]:
async def call_db(filename):
    conn = duckdb.connect(filename, read_only=True)
    query = """SELECT * 
           FROM candles
        """
    query_results = conn.execute(query).fetchall()

    conn.close()

    return query_results




In [5]:
query_data = await call_db(filename)  # ✅ If already in an async function


IOException: IO Error: File is already open in 
C:\Users\trrallele\AppData\Local\anaconda3\python.exe (PID 27556)

In [ ]:
#loop = asyncio.get_event_loop()
#query_data = loop.create_task(call_db(filename))  # ✅ Fire and forget


In [ ]:
#query_data = call_db(filename)

In [ ]:
print(query_data)

In [ ]:
#con = duckdb.connect(filename, read_only=True)
#print(con.execute("SELECT * FROM candles ORDER BY timestamp DESC LIMIT 5").fetchall())
#query = """SELECT * 
#           FROM candles
#        """
#query_results = con.execute(query).fetchall()

#con.close()

___

# Pipe the results to a Pandas Dataframe

In [ ]:
df = pd.DataFrame(query_data)

In [ ]:
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']

In [ ]:
df.tail(30)

In [ ]:
len(df)

____


____

___

In [ ]:
# strategy/ta.py
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import duckdb
import pandas as pd
import pandas_ta as ta
from datetime import datetime, timedelta

DB_PATH = filename  

def load_recent_candles(hours=200):
    con = duckdb.connect(DB_PATH)
    query = f"""
            SELECT * 
            FROM candles
            --WHERE timestamp >= NOW() - INTERVAL {hours} HOUR
            ORDER BY timestamp ASC
            """
    df = con.execute(query).fetchdf()
    con.close()
    return df


def compute_indicators(df):
    df.set_index('timestamp', inplace=True)

    # Add 3 basic indicators
    df['rsi'] = ta.rsi(df['close'], length=14)
    df['ema_50'] = ta.ema(df['close'], length=50)
    df['ema_200'] = ta.ema(df['close'], length=200)

    return df

def generate_signal(df):
    latest = df.iloc[-1]

    signal = "hold"
    reason = ""

    # Strategy logic
    if latest['ema_50'] > latest['ema_200'] and latest['rsi'] < 30:
        signal = "buy"
        reason = "Golden cross + RSI oversold"
    elif latest['ema_50'] < latest['ema_200'] and latest['rsi'] > 70:
        signal = "sell"
        reason = "Death cross + RSI overbought"

    return signal, reason

def get_ta_signal():
    df = load_recent_candles()
    df = compute_indicators(df)
    signal, reason = generate_signal(df)
    print(f"TA Signal: {signal.upper()} | Reason: {reason}")
    return signal, reason


In [ ]:
load_recent_candles()

In [ ]:
# Define the checkmark symbol using Unicode
checkmark = "\u2705"

# Display it to simulate a live conversion/output in a Python environment
checkmark
